## Inbeddingen

In ons vorige voorbeeld werkten we met hoog-dimensionale bag-of-words vectoren met een lengte van `vocab_size`, en we hebben expliciet laag-dimensionale positionele representatievectoren omgezet in een spaarzame one-hot representatie. Deze one-hot representatie is niet geheugen-efficiënt. Bovendien wordt elk woord onafhankelijk van de andere behandeld, waardoor one-hot gecodeerde vectoren geen semantische overeenkomsten tussen woorden uitdrukken.

In deze eenheid gaan we verder met het verkennen van de **News AG** dataset. Om te beginnen, laten we de data laden en enkele definities uit de vorige eenheid ophalen.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Wat is een embedding?

Het idee van een **embedding** is om woorden te representeren met lagere-dimensionale, dense vectoren die de semantische betekenis van het woord weerspiegelen. Later zullen we bespreken hoe je betekenisvolle woordembeddings kunt bouwen, maar voor nu kun je embeddings zien als een manier om de dimensionaliteit van een woordvector te verminderen.

Een embedding-laag neemt een woord als invoer en produceert een uitvoervector met een gespecificeerde `embedding_size`. In zekere zin lijkt het erg op een `Dense`-laag, maar in plaats van een one-hot encoded vector als invoer te nemen, kan het een woordnummer verwerken.

Door een embedding-laag als de eerste laag in ons netwerk te gebruiken, kunnen we overschakelen van een bag-of-words-model naar een **embedding bag**-model, waarbij we eerst elk woord in onze tekst omzetten naar de bijbehorende embedding en vervolgens een aggregatiefunctie toepassen op al deze embeddings, zoals `sum`, `average` of `max`.

![Afbeelding die een embedding-classificator toont voor vijf sequentiewoorden.](../../../../../translated_images/nl/embedding-classifier-example.b77f021a7ee67eee.webp)

Ons classifier-neuraal netwerk bestaat uit de volgende lagen:

* `TextVectorization`-laag, die een string als invoer neemt en een tensor van tokennummers produceert. We zullen een redelijke woordenschatgrootte `vocab_size` specificeren en minder vaak gebruikte woorden negeren. De invoervorm zal 1 zijn, en de uitvoervorm zal $n$ zijn, omdat we $n$ tokens als resultaat krijgen, elk met nummers van 0 tot `vocab_size`.
* `Embedding`-laag, die $n$ nummers neemt en elk nummer reduceert tot een dense vector van een gegeven lengte (100 in ons voorbeeld). Dus de invoertensor van vorm $n$ wordt getransformeerd naar een $n\times 100$-tensor.
* Aggregatielaag, die het gemiddelde van deze tensor berekent langs de eerste as, d.w.z. het zal het gemiddelde berekenen van alle $n$ invoertensors die overeenkomen met verschillende woorden. Om deze laag te implementeren, gebruiken we een `Lambda`-laag en geven we de functie door om het gemiddelde te berekenen. De uitvoer zal een vorm van 100 hebben en het zal de numerieke representatie van de hele invoersequentie zijn.
* De laatste `Dense` lineaire classificator.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


In de `samenvatting`-uitvoer, in de **outputvorm**-kolom, komt de eerste tensor-dimensie `None` overeen met de minibatch-grootte, en de tweede komt overeen met de lengte van de tokenreeks. Alle tokenreeksen in de minibatch hebben verschillende lengtes. We zullen in de volgende sectie bespreken hoe hiermee om te gaan.

Laten we nu het netwerk trainen:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Opmerking** dat we een vectorizer bouwen op basis van een subset van de gegevens. Dit wordt gedaan om het proces te versnellen, en het kan resulteren in een situatie waarin niet alle tokens uit onze tekst in de woordenschat aanwezig zijn. In dat geval worden die tokens genegeerd, wat kan leiden tot een iets lagere nauwkeurigheid. Echter, in de praktijk geeft een subset van tekst vaak een goede schatting van de woordenschat.


### Omgaan met variabele sequentiegroottes

Laten we begrijpen hoe training plaatsvindt in minibatches. In het bovenstaande voorbeeld heeft de invoertensor dimensie 1, en we gebruiken minibatches van 128, zodat de werkelijke grootte van de tensor $128 \times 1$ is. Echter, het aantal tokens in elke zin is verschillend. Als we de `TextVectorization`-laag toepassen op een enkele invoer, is het aantal geretourneerde tokens verschillend, afhankelijk van hoe de tekst wordt getokeniseerd:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Echter, wanneer we de vectorizer toepassen op meerdere sequenties, moet deze een tensor van rechthoekige vorm produceren, dus vult hij ongebruikte elementen met de PAD-token (wat in ons geval nul is):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Hier kunnen we de embeddings zien:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Opmerking**: Om de hoeveelheid opvulling te minimaliseren, is het in sommige gevallen logisch om alle reeksen in de dataset te sorteren op toenemende lengte (of, meer precies, aantal tokens). Dit zorgt ervoor dat elke minibatch reeksen van vergelijkbare lengte bevat.


## Semantische embeddings: Word2Vec

In ons vorige voorbeeld leerde de embeddinglaag om woorden naar vectorrepresentaties te mappen, maar deze representaties hadden geen semantische betekenis. Het zou handig zijn om een vectorrepresentatie te leren waarbij vergelijkbare woorden of synoniemen overeenkomen met vectoren die dicht bij elkaar liggen in termen van een bepaalde vectordistantie (bijvoorbeeld euclidische afstand).

Om dat te bereiken, moeten we ons embeddingmodel vooraf trainen op een grote verzameling tekst met behulp van een techniek zoals [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Dit is gebaseerd op twee hoofdarchitecturen die worden gebruikt om een gedistribueerde representatie van woorden te produceren:

 - **Continuous bag-of-words** (CBoW), waarbij we het model trainen om een woord te voorspellen op basis van de omliggende context. Gegeven de ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$, is het doel van het model om $W_0$ te voorspellen op basis van $(W_{-2},W_{-1},W_1,W_2)$.
 - **Continuous skip-gram** is het tegenovergestelde van CBoW. Het model gebruikt het omliggende venster van contextwoorden om het huidige woord te voorspellen.

CBoW is sneller, terwijl skip-gram langzamer is, maar beter presteert bij het representeren van zeldzame woorden.

![Afbeelding die zowel de CBoW- als Skip-Gram-algoritmen toont om woorden naar vectoren om te zetten.](../../../../../translated_images/nl/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Om te experimenteren met de Word2Vec-embedding die vooraf is getraind op de Google News-dataset, kunnen we de **gensim**-bibliotheek gebruiken. Hieronder vinden we de woorden die het meest lijken op 'neural'.

> **Opmerking:** Wanneer je voor het eerst woordvectoren aanmaakt, kan het downloaden ervan enige tijd in beslag nemen!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


We kunnen ook de vectorembedding uit het woord extraheren, om te gebruiken bij het trainen van het classificatiemodel. De embedding heeft 300 componenten, maar hier laten we alleen de eerste 20 componenten van de vector zien voor duidelijkheid:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Het geweldige aan semantische embeddings is dat je de vectorcodering kunt manipuleren op basis van semantiek. Bijvoorbeeld, we kunnen vragen om een woord te vinden waarvan de vectorrepresentatie zo dicht mogelijk bij de woorden *koning* en *vrouw* ligt, en zo ver mogelijk van het woord *man*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Een voorbeeld hierboven gebruikt wat interne GenSym-magie, maar de onderliggende logica is eigenlijk vrij eenvoudig. Een interessant aspect van embeddings is dat je normale vectorbewerkingen kunt uitvoeren op embeddingvectoren, en dat zou bewerkingen op woord**betekenissen** weerspiegelen. Het bovenstaande voorbeeld kan worden uitgedrukt in termen van vectorbewerkingen: we berekenen de vector die overeenkomt met **KONING-MAN+VROUW** (bewerkingen `+` en `-` worden uitgevoerd op vectorrepresentaties van overeenkomstige woorden), en vinden vervolgens het dichtstbijzijnde woord in het woordenboek bij die vector:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: We moesten een kleine coëfficiënt toevoegen aan de *man*- en *vrouw*-vectoren - probeer ze te verwijderen om te zien wat er gebeurt.

Om de dichtstbijzijnde vector te vinden, gebruiken we TensorFlow-mechanismen om een vector van afstanden te berekenen tussen onze vector en alle vectoren in de woordenschat, en vervolgens vinden we de index van het kleinste woord met behulp van `argmin`.


Hoewel Word2Vec een geweldige manier lijkt om woordsemantiek uit te drukken, heeft het verschillende nadelen, waaronder de volgende:

* Zowel CBoW- als skip-grammodellen zijn **predictieve embeddings** en houden alleen rekening met lokale context. Word2Vec maakt geen gebruik van globale context.
* Word2Vec houdt geen rekening met de **morfologie** van woorden, oftewel het feit dat de betekenis van een woord kan afhangen van verschillende delen van het woord, zoals de stam.

**FastText** probeert de tweede beperking te overwinnen en bouwt voort op Word2Vec door vectorrepresentaties te leren voor elk woord en de karakter-n-grams die in elk woord voorkomen. De waarden van deze representaties worden vervolgens gemiddeld tot één vector bij elke trainingsstap. Hoewel dit veel extra rekenkracht toevoegt aan de pretraining, stelt het woordembeddings in staat om subwoordinformatie te coderen.

Een andere methode, **GloVe**, gebruikt een andere benadering voor woordembeddings, gebaseerd op de factorisatie van de woord-contextmatrix. Eerst bouwt het een grote matrix die het aantal woordvoorkomens in verschillende contexten telt, en vervolgens probeert het deze matrix in lagere dimensies te representeren op een manier die het reconstructieverlies minimaliseert.

De gensim-bibliotheek ondersteunt deze woordembeddings, en je kunt ermee experimenteren door de model-laadcode hierboven aan te passen.


## Gebruik van vooraf getrainde embeddings in Keras

We kunnen het bovenstaande voorbeeld aanpassen om de matrix in onze embeddinglaag vooraf te vullen met semantische embeddings, zoals Word2Vec. De woordenschat van de vooraf getrainde embedding en de tekstcorpus zullen waarschijnlijk niet overeenkomen, dus we moeten er één kiezen. Hier verkennen we de twee mogelijke opties: het gebruik van de tokenizer-woordenschat en het gebruik van de woordenschat van Word2Vec-embeddings.

### Gebruik van tokenizer-woordenschat

Bij het gebruik van de tokenizer-woordenschat zullen sommige woorden uit de woordenschat overeenkomende Word2Vec-embeddings hebben, terwijl andere ontbreken. Aangezien onze woordenschatgrootte `vocab_size` is en de lengte van de Word2Vec embeddingvector `embed_size` is, zal de embeddinglaag worden weergegeven door een gewichts-matrix met de vorm `vocab_size`$\times$`embed_size`. We vullen deze matrix door de woordenschat door te nemen:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Voor woorden die niet aanwezig zijn in de Word2Vec-woordenschat, kunnen we ze ofwel als nullen laten, of een willekeurige vector genereren.

Nu kunnen we een embeddinglaag definiëren met vooraf getrainde gewichten:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Opmerking**: Merk op dat we `trainable=False` instellen bij het maken van de `Embedding`, wat betekent dat we de Embedding-laag niet opnieuw trainen. Dit kan ervoor zorgen dat de nauwkeurigheid iets lager is, maar het versnelt de training.

### Gebruik van embedding vocabulaire

Een probleem met de vorige aanpak is dat de woordenschatten die worden gebruikt in de TextVectorization en Embedding verschillend zijn. Om dit probleem op te lossen, kunnen we een van de volgende oplossingen gebruiken:
* Het Word2Vec-model opnieuw trainen op onze woordenschat.
* Onze dataset laden met de woordenschat van het vooraf getrainde Word2Vec-model. Woordenschatten die worden gebruikt om de dataset te laden, kunnen tijdens het laden worden gespecificeerd.

De laatste aanpak lijkt eenvoudiger, dus laten we deze implementeren. Allereerst zullen we een `TextVectorization`-laag maken met de opgegeven woordenschat, afkomstig van de Word2Vec embeddings:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

De gensim word embeddings-bibliotheek bevat een handige functie, `get_keras_embeddings`, die automatisch de bijbehorende Keras embeddings-laag voor je zal aanmaken.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


Een van de redenen waarom we geen hogere nauwkeurigheid zien, is omdat sommige woorden uit onze dataset ontbreken in de voorgetrainde GloVe-woordenschat en daardoor in feite worden genegeerd. Om dit te verhelpen, kunnen we onze eigen embeddings trainen op basis van onze dataset.


## Contextuele embeddings

Een belangrijke beperking van traditionele vooraf getrainde embedding-representaties zoals Word2Vec is dat, hoewel ze enige betekenis van een woord kunnen vastleggen, ze geen onderscheid kunnen maken tussen verschillende betekenissen. Dit kan problemen veroorzaken in modellen die hierop voortbouwen.

Bijvoorbeeld, het woord 'play' heeft verschillende betekenissen in deze twee zinnen:
- Ik ging naar een **toneelstuk** in het theater.
- John wil **spelen** met zijn vrienden.

De vooraf getrainde embeddings waar we het over hadden, vertegenwoordigen beide betekenissen van het woord 'play' in dezelfde embedding. Om deze beperking te overwinnen, moeten we embeddings bouwen op basis van het **taalmodel**, dat is getraind op een grote hoeveelheid tekst en *weet* hoe woorden in verschillende contexten samen kunnen worden gebruikt. Het bespreken van contextuele embeddings valt buiten de scope van deze tutorial, maar we komen hierop terug wanneer we taalmodellen bespreken in de volgende eenheid.



---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in zijn oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor cruciale informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor eventuele misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
